# ·例1 求任一非负实数的平方根

问题描述：求出$ y*y=x $的非负实数$ y $

实际过程中，对于给定数值，仅包含有穷位小数，平方根通常为一无理数，因此一般仅能得到近似值。因此，问题可以修改为：对任意实数$ x $，设法找到一个非负实数$ y $，使得$ |y*y-x|<e $，其中$ e $是事先给定的允许误差

牛顿法求解上述问题:  
(1) 对给定正实数$ x $和允许误差$ e $，令变量$ y $取任意正实数值，如令$ y=x $；  
(2) 如果$ y*y $与$ x $足够接近，即$ |y*y-x|<e $，计算结束并把$ y $作为结果；  
(3) 去$ z=(y+x/y)/2 $;  
(4) 将 $ z $作为$ y $的新值，回到步骤(2)。


In [3]:
# 代码实现
def sqrt(x):
    y = 1.0
    while abs(y * y - x) > 1e-6:
        y = (y + x/y)/2
    return y

上述代码中，$ y $的初始值为1.0，允许误差为$ 10^{-6} $  
## ------------------------------------------------------------------------------------------------------------------------

# ·例2 交叉路口的红绿灯安排

问题描述：如图所示五条线的交叉口，其中两条路是单行线，其余是正常的双向行驶道路。忽略道路方位，道路宽度和车流量，只考虑红绿灯如何安排，可以保证相应的允许行驶路线互不交错，从而保证路口的交通安全。(AD和ED互不冲突)

<img src="./图/1.png",width = 200, height = 200>

基本思想：  
·同属一组的各个方向行驶的车辆，均可以正常行驶，不出现相互交错的行驶线路；  
·所做的分组尽可能大，以提高路口的效率

## 问题分析与严格化
统计可行方向:13个，分别为AB,AC,AD,BA,BC,BD,DA,DB,DC,EA,EB,EC,ED  
绘制**冲突图**如下所示，13个小矩形代表可能行驶的方向，两个矩形间有连线代表它们的行驶路线相互冲突。  

<img src="./图/2.png",width = 300, height = 300>
有了冲突图，寻找安全分组问题就可以表述为：为冲突图中的顶点确定一种分组，保证属于同一组的顶点互不邻接，同时希望分组最少。


## 图的顶点分组和算法
  
以非相邻为条件的最佳顶点分组问题，对应于著名的图最佳着色问题。四色问题表明，对于以任意方式分割为区域的平面图，只需要用4种不同颜色着色，就能保证相邻区域都极有不同的颜色。（需注意的是交叉路口情况构建的冲突图可能不是平面图，因此可能需要更多的颜色）最佳着色算法基本相当于枚举所有可能，算法代价高。因而可以采用贪婪算法来解决。  
贪婪算法的基本思想是根据掌握的信息，尽可能的向得到解的方向前进，知道不能继续再换一个方向。该方法通常不能得到最优解，但能找到“可接受的”解。算法的伪代码如下：    

    输入：图G              # 记录图中的顶点连接关系  
    集合verts保存G中所有顶点    # 建立初始状态  
    设置集合groups为空集       # 记录得到的分组，元素是集合顶点  
    while存在未着色顶点：      # 每次迭代用贪心发找一个新分组  
        选一种新颜色  
        在未着色顶点中给尽可能多的无互连边的点着色（构建一个分组）  
        记录新着色的顶点组  
    
    # 算法结束是集合groups里记录着一种分组方式  
    # 算法细节还需要进一步考虑
 
现在考虑用算法处理图中的冲突图，操作情况如下：  
(1) 选第1种颜色构造第1个分组：顺序选出互相不冲突的AB、AC、AD，以及与任何方向都不冲突的BA、DC和ED，做成一组；  
(2) 选出BC、BD和于他们不冲突的EA作为第二组；   
(3) 选出DA和DB作为第三组；  
(4) 剩下的EB和EC作为第四组。  

上述算法还未开了一种新颜色的着色处理，修改后为：  
    
    new_group = 空集  
    for v in verts:  
        if v与new_group中所有顶点间都没有边：  
            从verts中去掉v  
            把v加入new_group  
    
    # 循环结束时new_group是可以用一种新颜色着色的顶点集合  
    # 用这段代替前面程序框架中主循环体的一部分  
 

## 算法的精化和Python描述  
· 判断一个集合vs是空集，对应于直接判断not vs  
· 设置一个集合为空：vs = set()  
· 从集合中去掉元素v: vs.remove(v)  
· 集合中增加元素：vs.add(v)  
并假设两个函数：  
· 函数vertices(G)得到G中所有顶点的集合  
· not_adjacent_with_set(v,group,G)检查顶点c与顶点集group中各顶点在图G中是否有边连接

In [ ]:
# 程序
def coloring(G):
    color = 0
    groups = set()
    verts = vertices(G)
    while verts:
        new_group = set()
        for v in list(verts):
            if not_adjacent_with_set(v, newgroup, G):
                new_group.add()
                verts.remove(v)
        groups.add((color, new_group))
        color += 1
    return groups

## 讨论
解是否唯一？不唯一  
解应适当扩充，加入与已有成员不冲突的方向，得到：  
{AB, AC, AD, BA, DC, ED}, {BC, BD, EA, BA, DC, ED}, {DA, DB, BA, DC, ED, AD}, {EB, EC, BA, BC, ED, EA}  
## ------------------------------------------------------------------------------------------------------------------------

# ·算法复杂度  
常用： $ O(1), O(log n), O(n), O(n^2), O(n^3), O(2^n) $  
空间复杂度和时间复杂度类似  
  
## 解决同一个问题的不同算法  
例：求斐波那契数列的第n项。斐波那契数列定义：$ F_0 = F_1 = 1; F_n = F_{n-1} + F_{n-2}\quad for\; n>1 $

In [ ]:
# 求解方式1：递归
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

方式1：计算$ F_n $的时间代价(考虑求加法操作的次数)大致等于计算$ F_{n-1} $和$ F_{n-2} $的时间代价之和，等比于斐波那契数$ F_n $的值。根据已有结论，$ \lim_{n \to \infty}F_n\, = \, \left(\frac{\sqrt{5} + 1}{2} \right)^n \approx 1.618^n $，计算$ F_n $的时间代价按$ n $的指数增长

In [ ]:
# 求解方式2： 递推
def fib(n):
    f1 = f2 = 1
    for k in range(1,n):
        f1, f2 = f2, f2 + f1
    return f2

方式2：循环前工作一次，循环n-1次，总工作量和n呈线性关系  

## 算法分析
算法分析的目的是推导出算法的复杂度，最主要的技术是构造和求解递归函数。  

### 基本循环程序
基本计算规则：  
(1) 基本操作，认为其时间复杂度为O(1)。  
(2) 加法操作(顺序复合)： $ O(T(n))=O(T_1(n)+T_2(n))=O(max(T_1(n),T_2(n)) $  
(3) 乘法操作(循环结构)： $ O(T(n))=O(T_1(n)*O(T_2(n))=O(T_1(n)*T_2(n)) $  
(4) 取最大规则(分支结构)：$ O(T(n))=O(max(T_1(n),T_2(n)) $  

In [ ]:
# 例
for i in range(n):
    for j in range(n):
        x = 0.0
        for k in range(n):
            x = x + m1[i][k] + m2[k][j]
        m[i][j] = x

上述程序复杂度： $ O(T(n)) = O(n)*O(n)*(O(1) + O(n)*O(1) + O(1))=O(n^3) $  

## python 程序的计算代价(复杂度)
### 时间开销
常量时间开销：基本算术运算，逻辑运算；list元素访问与赋值(tuple元素访问)等  
非常量时间开销：复制和切片(O(n))；字符串操作；创建对象；构造新结构，如list，set(O(n))；list加入和删除元素(O(n))；  
字典操作：情况较为复杂，最坏情况为(O(n))，平均复杂度为(O(1))  

### 空间复杂度
表和元组：(O(n))，集合和字典结构更为复杂，至少需要(O(n))  
## ------------------------------------------------------------------------------------------------------------------------

# 数据结构
抽象定义：一个数据结构包含一集数据元素，是一个有穷集，在这些元素之间存在某些特定的逻辑关系。可表示为$ D=(E,R) $  
典型数据结构：  
·集合结构：其数据元素间没有明确的关系，即R是空集；  
·序列结构：数据元素之间有一种明确的顺序关系，关系R是线性顺序关系，这种结构也称为**线性结构**，序列结构还有一些变形，如环形结构和$\rho$形结构  
<img src="./图/3.png",width = 300, height = 300>
·层次结构  
·**树形结构**：最简单的一种层次结构  
·**图结构**：前面几种结构都可以称为图结构

## 计算机内存对象表示
### 内存单元和地址
内存是CPU可以直接访问的数据存储设备，与之对应的为外存，如磁盘等，保存在外存里的数据须装入内存才能被CPU访问  
内存的基本结构是线性排列的一批存储单元，每个单元大小相同，具体单元大小因计算机的不同而有所差异，内存单元具有唯一编号，称为**地址**，对内存单元的访问时通过单元的地址进行。基于地址访问内存单元时一个O(1)操作，与单元的位置或整个内存的大小无关。  

### 对象存储和管理
表示程序中的对象，需在空闲的内存中确定一块或几块区域，将对象的数据存入其中。当对象不再有用时，存储管理系统会回收对象占用的存储。对象的访问是O(1)操作  

### 对象关联的表示
顺序结构；链式结构  

python变量实现方式是引用语义：在变量里保存值(对象)的引用，C语言采用的是值语义：变量的值直接保存在变量的存储区  

